# 08

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163756 sha256=73c0d7c1176b7c7eb6b7b317fbac2c46872f056c04a6bbb38f6359732060898d
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split


In [3]:
data = Dataset.load_builtin('ml-100k')

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
algo = SVD()
algo.fit(trainset)

In [5]:
predictions = algo.test(testset)
print('prediction type:', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.583701469838746, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.7719507606375418, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.298990343301177, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6734355467476156, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.3501252899186564, details={'was_impossible': False})]

In [6]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.583701469838746),
 ('882', '291', 3.7719507606375418),
 ('535', '507', 4.298990343301177)]

In [7]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.00   {'was_impossible': False}


In [8]:
accuracy.rmse(predictions)

RMSE: 0.9501


0.9501009694934589

In [19]:
import pandas as pd
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/ratings.csv')

ratings.to_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/ratings_noh.csv', index=False, header=False)

In [23]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=', ', rating_scale=(0.5,5))


######
data = Dataset.load_from_file('/content/drive/MyDrive/Colab Notebooks/Datasets/ratings_noh.csv', reader = reader)

ValueError: Impossible to parse line. Check the line_format and sep parameters.

In [24]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9457


0.9456994032241722

In [26]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)
algo=SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

In [29]:
from surprise.model_selection import cross_validate

ragins = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8824  0.8742  0.8711  0.8685  0.8765  0.8746  0.0048  
MAE (testset)     0.6802  0.6720  0.6679  0.6675  0.6744  0.6724  0.0047  
Fit time          1.58    2.42    4.74    1.86    1.76    2.47    1.17    
Test time         0.30    0.33    0.40    0.13    0.15    0.26    0.10    


{'test_rmse': array([0.88244727, 0.87421043, 0.87113181, 0.86854619, 0.87648496]),
 'test_mae': array([0.68023143, 0.6719532 , 0.66786186, 0.66746061, 0.67444487]),
 'fit_time': (1.5826911926269531,
  2.4231183528900146,
  4.742773532867432,
  1.8636970520019531,
  1.7593491077423096),
 'test_time': (0.30329012870788574,
  0.32827043533325195,
  0.39516425132751465,
  0.1325671672821045,
  0.1533644199371338)}

In [33]:
from surprise.model_selection import GridSearchCV

param_grid ={
    'n_epochs':[20,40,60],
    'n_factors':[50,100,200]
}

gs = GridSearchCV(SVD, param_grid, measures=['RMSE','MAE'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8768673381228655
{'n_epochs': 20, 'n_factors': 50}


In [ ]:
data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

## 오류


In [34]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=', ', rating_scale=(0.5,5))
data_folds= DatasetAutoFolds(ratings_file='/content/drive/MyDrive/Colab Notebooks/Datasets/ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

ValueError: Impossible to parse line. Check the line_format and sep parameters.

In [35]:
algo=SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [36]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datasets/movies.csv')

movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count()==0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [37]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.50   {'was_impossible': False}


In [38]:
def get_unseen_surprise(ratings, movies, userId):
  seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
  total_movies = movies['movieId'].tolist()

  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies),  '추천 대상 영화 수:', len(unseen_movies),
        '정체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)


평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 정체 영화 수: 9742


In [42]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):

  predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  def sortkey_est(pred):
    return pred.est

  predictions.sort(key=sortkey_est, reverse=True)
  top_predictions = predictions[:top_n]

  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_ratings = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_ratings)]

  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
  print(top_movie[1], ":", top_movie[2])


평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 정체 영화 수: 9742
##### Top-10 추천 영화 리스트 #####
Toy Story (1995) : 3.5002644558160445
Jumanji (1995) : 3.5002644558160445
Grumpier Old Men (1995) : 3.5002644558160445
Waiting to Exhale (1995) : 3.5002644558160445
Father of the Bride Part II (1995) : 3.5002644558160445
Heat (1995) : 3.5002644558160445
Sabrina (1995) : 3.5002644558160445
Tom and Huck (1995) : 3.5002644558160445
Sudden Death (1995) : 3.5002644558160445
GoldenEye (1995) : 3.5002644558160445
